<a href="https://colab.research.google.com/github/atjoelpark/eutilities/blob/main/demo/Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of Data

1. Extraction of first names
2. Utilization of the Genderize API
3. Identification of Country

In [1]:
# Customize here
API_KEY = ""
GOOGLE_DRIVE_URL = ""
GOOGLE_FOLDER_TO_SAVE = "Research"

In [2]:
# Mounting Google Drive to Google Colab
from google.colab import drive
drive.mount(f'/content/drive/{GOOGLE_DRIVE_URL}')

Mounted at /content/drive/


In [3]:
# Installing dependencies
import numpy as np 
import pandas as pd 
import re 

# For Logging & Debugging: Not needed for analysis
import logging

In [4]:
# Installing Genderize
!pip install genderize

In [5]:
# Reference: https://genderize.io
# Reference: https://github.com/SteelPangolin/genderize
# Importing Genderize
from genderize import Genderize

# Methods

In [6]:
class Country:

    # This class produces several static methods that assist in the search phase including utilizing APIs for Genderize as well as NLM's website. No fields are declared nor constructors instantiated. As a result, any import of the Search class will not require an instantiation. There will however be some static field types that wil help faciliate some of the static methods here.

    def __init__(self):
        self.throwAway = 1

    @staticmethod
    def countryCount(arrayOfAffiliations):
        # This returns a dictionary with a dictionary with key = country and value = frequency of the country. There is one parameters with the first parameter ingesting a string of affiliations. 

        # Libraries utilized
        from collections import Counter
        import pandas as pd

        # Declare an array with list of all valid countries in the world.
        # Reference: https://textlists.info/geography/countries-of-the-world/
        listOfCountries = ["afghanistan", "albania", "algeria", "andorra", "angola", "antigua and barbuda", "argentina", "armenia", "australia", "austria", "azerbaijan", "the bahamas", "bahrain", "bangladesh", "barbados", "belarus", "belgium", "belize", "benin", "bhutan", "bolivia", "bosnia and herzegovina", "botswana", "brazil", "brunei", "bulgaria", "burkina faso", "burundi", "cambodia", "cameroon", "canada", "cape verde", "central african republic", "chad", "chile", "china", "colombia", "comoros", "congo, republic of the", "congo, democratic republic of the", "costa rica", "cote d'ivoire", "croatia", "cuba", "cyprus", "czech republic", "denmark", "djibouti", "dominica", "dominican republic", "east timor (timor-leste)", "ecuador", "egypt", "el salvador", "equatorial guinea", "eritrea", "estonia", "ethiopia", "fiji", "finland", "france","gabon","the gambia","georgia","germany","ghana","greece","grenada","guatemala","guinea","guinea-bissau","guyana","haiti","honduras","hungary","iceland","india","indonesia","iran","iraq","ireland","israel","italy","jamaica","japan","jordan","kazakhstan","kenya","kiribati","korea,north","korea, south","kosovo","kuwait","kyrgyzstan","laos","latvia","lebanon","lesotho","liberia","libya","liechtenstein","lithuania","luxembourg","macedonia","madagascar","malawi","malaysia","maldives","mali","malta","marshall islands","mauritania","mauritius","mexico","micronesia, federated states of","moldova","monaco","mongolia","montenegro","morocco","mozambique","myanmar (burma)","namibia","nauru","nepal","netherlands","new zealand","nicaragua","niger","nigeria","norway","oman","pakistan","palau","panama","papua new guinea","paraguay","peru","philippines","poland","portugal","qatar","romania","russia","rwanda","saint kitts and nevis","saint lucia","saint vincent and the grenadines","samoa","san marino","sao tome and principe","saudi arabia","senegal","serbia","seychelles","sierra leone","singapore","slovakia","slovenia","solomon islands","somalia","south africa","south sudan","spain","sri lanka","sudan","suriname","swaziland","sweden","switzerland","syria","taiwan","tajikistan","tanzania","thailand","togo","tonga","trinidad and tobago","tunisia","turkey","turkmenistan","tuvalu","uganda","ukraine","united arab emirates","united kingdom","united states of america","uruguay","uzbekistan","vanuatu","vatican city (holy see)","venezuela","vietnam","yemen","zambia","zimbabwe"]
        punc = '''!|()-[]{};:'"\, <>./?@#$%^&*_~'''

        # Please note that some countries will require mapping
        # i.e. "South Korea" --> "Korea, South"
        # Please note that I account for these edge cases in the main body of the method below
        # This may require further iterations to improve on these edge cases
        # punctuation has also been declared so that they can be removed

        # TODO: Map the country initials to the country's full name. https://datahub.io/core/country-list
        abbrev_dict = {
                "Afghanistan":"AF",
                "Åland Islands":"AX",
                "Albania":"AL",
                "Algeria":"DZ",
                "American Samoa":"AS",
                "Andorra":"AD",
                "Angola":"AO",
                "Anguilla":"AI",
                "Antarctica":"AQ",
                "Antigua and Barbuda":"AG",
                "Argentina":"AR",
                "Armenia":"AM",
                "Aruba":"AW",
                "Australia":"AU",
                "Austria":"AT",
                "Azerbaijan":"AZ",
                "Bahamas":"BS",
                "Bahrain":"BH",
                "Bangladesh":"BD",
                "Barbados":"BB",
                "Belarus":"BY",
                "Belgium":"BE",
                "Belize":"BZ",
                "Benin":"BJ",
                "Bermuda":"BM",
                "Bhutan":"BT",
                "Bolivia, Plurinational State of":"BO",
                "Bonaire, Sint Eustatius and Saba":"BQ",
                "Bosnia and Herzegovina":"BA",
                "Botswana":"BW",
                "Bouvet Island":"BV",
                "Brazil":"BR",
                "British Indian Ocean Territory":"IO",
                "Brunei Darussalam":"BN",
                "Bulgaria":"BG",
                "Burkina Faso":"BF",
                "Burundi":"BI",
                "Cambodia":"KH",
                "Cameroon":"CM",
                "Canada":"CA",
                "Cape Verde":"CV",
                "Cayman Islands":"KY",
                "Central African Republic":"CF",
                "Chad":"TD",
                "Chile":"CL",
                "China":"CN",
                "Christmas Island":"CX",
                "Cocos (Keeling) Islands":"CC",
                "Colombia":"CO",
                "Comoros":"KM",
                "Congo":"CG",
                "Congo, the Democratic Republic of the":"CD",
                "Cook Islands":"CK",
                "Costa Rica":"CR",
                "Côte d'Ivoire":"CI",
                "Croatia":"HR",
                "Cuba":"CU",
                "Curaçao":"CW",
                "Cyprus":"CY",
                "Czech Republic":"CZ",
                "Denmark":"DK",
                "Djibouti":"DJ",
                "Dominica":"DM",
                "Dominican Republic":"DO",
                "Ecuador":"EC",
                "Egypt":"EG",
                "El Salvador":"SV",
                "Equatorial Guinea":"GQ",
                "Eritrea":"ER",
                "Estonia":"EE",
                "Ethiopia":"ET",
                "Falkland Islands (Malvinas)":"FK",
                "Faroe Islands":"FO",
                "Fiji":"FJ",
                "Finland":"FI",
                "France":"FR",
                "French Guiana":"GF",
                "French Polynesia":"PF",
                "French Southern Territories":"TF",
                "Gabon":"GA",
                "Gambia":"GM",
                "Georgia":"GE",
                "Germany":"DE",
                "Ghana":"GH",
                "Gibraltar":"GI",
                "Greece":"GR",
                "Greenland":"GL",
                "Grenada":"GD",
                "Guadeloupe":"GP",
                "Guam":"GU",
                "Guatemala":"GT",
                "Guernsey":"GG",
                "Guinea":"GN",
                "Guinea-Bissau":"GW",
                "Guyana":"GY",
                "Haiti":"HT",
                "Heard Island and McDonald Islands":"HM",
                "Holy See (Vatican City State)":"VA",
                "Honduras":"HN",
                "Hong Kong":"HK",
                "Hungary":"HU",
                "Iceland":"IS",
                "India":"IN",
                "Indonesia":"ID",
                "Iran, Islamic Republic of":"IR",
                "Iraq":"IQ",
                "Ireland":"IE",
                "Isle of Man":"IM",
                "Israel":"IL",
                "Italy":"IT",
                "Jamaica":"JM",
                "Japan":"JP",
                "Jersey":"JE",
                "Jordan":"JO",
                "Kazakhstan":"KZ",
                "Kenya":"KE",
                "Kiribati":"KI",
                "Korea, Democratic People's Republic of":"KP",
                "Korea, Republic of":"KR",
                "Kuwait":"KW",
                "Kyrgyzstan":"KG",
                "Lao People's Democratic Republic":"LA",
                "Latvia":"LV",
                "Lebanon":"LB",
                "Lesotho":"LS",
                "Liberia":"LR",
                "Libya":"LY",
                "Liechtenstein":"LI",
                "Lithuania":"LT",
                "Luxembourg":"LU",
                "Macao":"MO",
                "Macedonia, the Former Yugoslav Republic of":"MK",
                "Madagascar":"MG",
                "Malawi":"MW",
                "Malaysia":"MY",
                "Maldives":"MV",
                "Mali":"ML",
                "Malta":"MT",
                "Marshall Islands":"MH",
                "Martinique":"MQ",
                "Mauritania":"MR",
                "Mauritius":"MU",
                "Mayotte":"YT",
                "Mexico":"MX",
                "Micronesia, Federated States of":"FM",
                "Moldova, Republic of":"MD",
                "Monaco":"MC",
                "Mongolia":"MN",
                "Montenegro":"ME",
                "Montserrat":"MS",
                "Morocco":"MA",
                "Mozambique":"MZ",
                "Myanmar":"MM",
                "Namibia":"NA",
                "Nauru":"NR",
                "Nepal":"NP",
                "Netherlands":"NL",
                "New Caledonia":"NC",
                "New Zealand":"NZ",
                "Nicaragua":"NI",
                "Niger":"NE",
                "Nigeria":"NG",
                "Niue":"NU",
                "Norfolk Island":"NF",
                "Northern Mariana Islands":"MP",
                "Norway":"NO",
                "Oman":"OM",
                "Pakistan":"PK",
                "Palau":"PW",
                "Palestine, State of":"PS",
                "Panama":"PA",
                "Papua New Guinea":"PG",
                "Paraguay":"PY",
                "Peru":"PE",
                "Philippines":"PH",
                "Pitcairn":"PN",
                "Poland":"PL",
                "Portugal":"PT",
                "Puerto Rico":"PR",
                "Qatar":"QA",
                "Réunion":"RE",
                "Romania":"RO",
                "Russian Federation":"RU",
                "Rwanda":"RW",
                "Saint Barthélemy":"BL",
                "Saint Helena, Ascension and Tristan da Cunha":"SH",
                "Saint Kitts and Nevis":"KN",
                "Saint Lucia":"LC",
                "Saint Martin (French part)":"MF",
                "Saint Pierre and Miquelon":"PM",
                "Saint Vincent and the Grenadines":"VC",
                "Samoa":"WS",
                "San Marino":"SM",
                "Sao Tome and Principe":"ST",
                "Saudi Arabia":"SA",
                "Senegal":"SN",
                "Serbia":"RS",
                "Seychelles":"SC",
                "Sierra Leone":"SL",
                "Singapore":"SG",
                "Sint Maarten (Dutch part)":"SX",
                "Slovakia":"SK",
                "Slovenia":"SI",
                "Solomon Islands":"SB",
                "Somalia":"SO",
                "South Africa":"ZA",
                "South Georgia and the South Sandwich Islands":"GS",
                "South Sudan":"SS",
                "Spain":"ES",
                "Sri Lanka":"LK",
                "Sudan":"SD",
                "Suriname":"SR",
                "Svalbard and Jan Mayen":"SJ",
                "Swaziland":"SZ",
                "Sweden":"SE",
                "Switzerland":"CH",
                "Syrian Arab Republic":"SY",
                "Taiwan":"TW",
                "Tajikistan":"TJ",
                "Tanzania":"TZ",
                "Thailand":"TH",
                "Timor-Leste":"TL",
                "Togo":"TG",
                "Tokelau":"TK",
                "Tonga":"TO",
                "Trinidad and Tobago":"TT",
                "Tunisia":"TN",
                "Turkey":"TR",
                "Turkmenistan":"TM",
                "Turks and Caicos Islands":"TC",
                "Tuvalu":"TV",
                "Uganda":"UG",
                "Ukraine":"UA",
                "United Arab Emirates":"AE",
                "United Kingdom":"GB",
                "United States of America":"US",
                "United States Minor Outlying Islands":"UM",
                "Uruguay":"UY",
                "Uzbekistan":"UZ",
                "Vanuatu":"VU",
                "Venezuela":"VE",
                "Vietnam":"VN",
                "Virgin Islands, British":"VG",
                "Virgin Islands, U.S.":"VI",
                "Wallis and Futuna":"WF",
                "Western Sahara":"EH",
                "Yemen":"YE",
                "Zambia":"ZM",
                "Zimbabwe":"ZW"}

        abbrev = pd.DataFrame(list(abbrev_dict.items()), columns=["Name", "Code"])

        affiliationsSplit = arrayOfAffiliations.split(" ")
        tempList = []                                            

        for word in affiliationsSplit:                           
            
            # Removing punctuations in string 
            # Using loop + punctuation string 
            for ele in word:  
                if ele in punc:  
                    word = word.replace(ele, "")  
                                                                   
            if word.lower() in listOfCountries: 
                tempList.append(word.lower())
            if word.lower() == "south korea": 
                tempList.append("korea, south") # edge case for South Korea
            if word.lower() == "north korea": 
                tempList.append("korea, north") # edge case for North Korea
            if word.lower() == "united states": 
                tempList.append("united states of america") # edge case for US
            if word.lower() == "usa": 
                tempList.append("united states of america") # edge case for US
            if word.lower() == "uk": 
                tempList.append("united kingdom") # edge case for US

            #if abbrev.loc[:,"Code"].str.contains(word.upper()).any():
            #    wordTemp = abbrev[abbrev.loc[:,"Code"] == word.upper()].loc[:,"Name"][0]
            #    tempList.append(wordTemp) # accounts for country abbreviations
            # BUG: Please look again for this error

        # List of countries established. Counter function returns a dictionary with country frequency
        return Counter(tempList)


class Tokenization():
    # This class provides static functions in tokenization and frequency counts of words.
    # No constructors are required.

    @staticmethod
    def return_tokens(index: int, aff_string: str) -> list:
        try:
            '''
            Returns a sorted list of words in descending order
            Dependency: nltk.tokenize, nltk.corpus
            '''
            from nltk.tokenize import RegexpTokenizer
            from nltk.corpus import stopwords

            tokenizer = RegexpTokenizer(r'\w+')
            tokens = tokenizer.tokenize(aff_string)
            tokens = [word.lower() for word in tokens]
            return [word for word in tokens if word not in stopwords.words('english')]
        except:
            # Occasionally the input creates a null.
            # Return an empty list 
            # print(f'Null encountered at index {i}')
            return []

    @staticmethod
    def return_frequency_terms(terms: list) -> dict:
        '''
        Returns a dictionary of the terms with its corresponding frequencies
        '''
        wordfreq = [terms.count(w) for w in terms]
        return dict(list(zip(terms,wordfreq)))

    @staticmethod
    def return_sorted_frequency(freqdict: dict) -> list:
        '''
        Returns a sorted list with the most frequent terms
        '''
        aux = [(freqdict[key], key) for key in freqdict]
        aux.sort()
        aux.reverse()
        return aux

In [13]:
def get_Gender(list_of_first_names: list, api_key = API_KEY) -> pd.DataFrame:
  """
  Ingests a list of first names and outputs a pandas dataframe with first name, probability and gender
  Please note that API key is optional. The API Key allows a higher limit of calls.
  A subscription is required in order to obtain an API key.
  Go to genderize.io to pay for a subscription. 
  """
  try:
    if api_key == "":
      genderize = Genderize(
          user_agent='GenderizeDocs/0.0',
          timeout=5.0
      )
    else:
      genderize = Genderize(
          user_agent='GenderizeDocs/0.0',
          api_key=api_key,
          timeout=5.0
      )
  except Exception as e:
    logger = logging.getLogger()
    logger.setLevel(logging.ERROR)
    logging.error(e)
    raise ValueError("Entered an invalid API KEY")

  list_probabilities: list = genderize.get(list_of_first_names)
  return pd.DataFrame(list_probabilities)

# Main

Let's start importing in the sample dataset and analyze the data.

In [22]:
sample_pmid = pd.read_csv('/content/drive/My Drive/Research/PMID_MetaData.csv')

In [24]:
# Inspect the dataframe
sample_pmid

,PMID,PubMed_Article_Title,Date_Completed_Year,Date_Completed_Month,Date_Completed_Day,Date_Revised_Year,Date_Revised_Month,Date_Revised_Day,Journal_Title,Publication_Date_Year,Publication_Date_Month,Publication_Date_Day,Abstract,AuthorFirstName_AuthorLastName_Affiliation
0,19008416,Genomic loss of microRNA-101 leads to overexpr...,2009,1,5,2018,11,13,"Science (New York, N.Y.)",2008,Dec,12,Enhancer of zeste homolog 2 (EZH2) is a mammal...,Sooryanarayana_Varambally_Michigan Center for ...
1,18927361,Genetics. DNA test for breast cancer risk draw...,2008,11,3,2009,11,19,"Science (New York, N.Y.)",2008,Oct,17,NaN,Jennifer_Couzin
2,18787170,FBXW7 targets mTOR for degradation and coopera...,2008,9,25,2018,11,13,"Science (New York, N.Y.)",2008,Sep,12,The enzyme mTOR (mammalian target of rapamycin...,"Jian-Hua_Mao_Cancer Research Institute, Univer..."
3,18487186,Design logic of a cannabinoid receptor signali...,2008,5,27,2018,11,13,"Science (New York, N.Y.)",2008,May,16,Cannabinoid receptor 1 (CB1R) regulates neuron...,Kenneth D_Bromberg_Department of Pharmacology ...
4,18239126,Cancer proliferation gene discovery through fu...,2008,2,12,2019,1,9,"Science (New York, N.Y.)",2008,Feb,1,Retroviral short hairpin RNA (shRNA)-mediated ...,Michael R_Schlabach_Howard Hughes Medical Inst...
5,18239125,Profiling essential genes in human mammary cel...,2008,2,12,2018,11,13,"Science (New York, N.Y.)",2008,Feb,1,By virtue of their accumulated genetic alterat...,Jose M_Silva_Watson School of Biological Scien...


In [34]:
# Inspect the first row and the cell of authors with their affiliations
sample_pmid.loc[0,"AuthorFirstName_AuthorLastName_Affiliation"]

'Sooryanarayana_Varambally_Michigan Center for Translational Pathology, University of Michigan Medical School, Ann Arbor, MI 48109, USA./Qi_Cao/Ram-Shankar_Mani/Sunita_Shankar/Xiaosong_Wang/Bushra_Ateeq/Bharathi_Laxman/Xuhong_Cao/Xiaojun_Jing/Kalpana_Ramnarayanan/J Chad_Brenner/Jindan_Yu/Jung H_Kim/Bo_Han/Patrick_Tan/Chandan_Kumar-Sinha/Robert J_Lonigro/Nallasivam_Palanisamy/Christopher A_Maher/Arul M_Chinnaiyan'

Inspecting this column, it appears that the delimiter is "/" with the first author having additional affiliation associated with the author 

For now, let's extract out all of the authors' first name.

In [43]:
# Extracting out authors' first names for all 6 papers using regular expressions
list_of_first_names = []
for counter in range(0,6,1):
  authors = sample_pmid.loc[counter,"AuthorFirstName_AuthorLastName_Affiliation"]
  list_authors = authors.split('/')
  
  for author in list_authors:
    first_name = re.findall(r'^[\s\w-]+?_',author) # Searches for first name including '-' and ' '
    first_name = re.sub(r'_','',first_name[0]) # Removes '_'
    first_name = re.findall(r'^[\w]+',first_name) # Many names have middle initials in it. This eliminates middle initial
    list_of_first_names.append(first_name[0])

In [44]:
# Obtain probabilities and gender of first names
first_names_df = get_Gender(list_of_first_names)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.genderize.io:443
DEBUG:urllib3.connectionpool:https://api.genderize.io:443 "GET /?name%5B%5D=Sooryanarayana&name%5B%5D=Qi&name%5B%5D=Ram&name%5B%5D=Sunita&name%5B%5D=Xiaosong&name%5B%5D=Bushra&name%5B%5D=Bharathi&name%5B%5D=Xuhong&name%5B%5D=Xiaojun&name%5B%5D=Kalpana HTTP/1.1" 200 656
DEBUG:urllib3.connectionpool:https://api.genderize.io:443 "GET /?name%5B%5D=J&name%5B%5D=Jindan&name%5B%5D=Jung&name%5B%5D=Bo&name%5B%5D=Patrick&name%5B%5D=Chandan&name%5B%5D=Robert&name%5B%5D=Nallasivam&name%5B%5D=Christopher&name%5B%5D=Arul HTTP/1.1" 200 654
DEBUG:urllib3.connectionpool:https://api.genderize.io:443 "GET /?name%5B%5D=Jennifer&name%5B%5D=Jian&name%5B%5D=Il&name%5B%5D=Di&name%5B%5D=Joan&name%5B%5D=Hio&name%5B%5D=Reyno&name%5B%5D=Allan&name%5B%5D=Kenneth&name%5B%5D=Avi HTTP/1.1" 200 647
DEBUG:urllib3.connectionpool:https://api.genderize.io:443 "GET /?name%5B%5D=Susana&name%5B%5D=Ravi&name%5B%5D=Michael&name%5B%5D=Ji&name%5

In [45]:
# Inspecting head of data frame
first_names_df.head()

,name,gender,probability,count
0,Sooryanarayana,None,0.00,0
1,Qi,male,0.64,445
2,Ram,male,0.93,5003
3,Sunita,female,0.97,695
4,Xiaosong,male,0.83,12


In [46]:
# Shape of the dataframe
first_names_df.shape

(58, 4)

In [47]:
# Saving first_names_df
first_names_df.to_csv('/content/drive/My Drive/Research/first_names_df.csv', index=False)